# Пример использования сценария

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K.

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

## Загрузка данных <a name="data-loader"></a>

In [3]:
# используем пакет https://pypi.org/project/rs-datasets/
!pip install rs_datasets

In [4]:
from rs_datasets import MovieLens

data = MovieLens("100k")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [5]:
# загрузим данные в Spark

from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [6]:
from sponge_bob_magic.models import ALSWrap, KNN, LightFMWrap, PopRec

pop_rec = PopRec()
als_rec = ALSWrap()
knn_rec = KNN()
lightfm_rec = LightFMWrap()

In [7]:
from sponge_bob_magic.splitters import UserSplitter, RandomSplitter, DateSplitter, ColdUsersSplitter
from datetime import datetime

log_bydate_splitter = DateSplitter(
    test_start=datetime(2007, 1, 1),
    drop_cold_items=True,
    drop_cold_users=True
)
log_random_splitter = RandomSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)
log_cold_splitter = ColdUsersSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True
)
user_random_splitter = UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
    
)
user_bydate_splitter = UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [8]:
from sponge_bob_magic.scenarios import MainScenario
from sponge_bob_magic.metrics import NDCG, Precision, MAP, Recall, Surprisal, HitRate

scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=lightfm_rec,
    criterion=HitRate,
    metrics={
        NDCG: [10, 5, 3],
        Surprisal: [10, 5, 3],
    },
    fallback_rec=pop_rec,
)

In [9]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [10]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

24-Apr-20 14:30:23, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
24-Apr-20 14:30:30, sponge_bob_magic, DEBUG: Длина трейна и теста: 84283 15690
24-Apr-20 14:30:32, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 943, 500
24-Apr-20 14:30:36, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 1657, 1334
24-Apr-20 14:30:36, sponge_bob_magic, DEBUG: Инициализация метрик
24-Apr-20 14:30:37, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
24-Apr-20 14:30:37, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
24-Apr-20 14:30:38, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
/home/boris/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning:

pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead

24-Apr-20 14:30:44, sponge_bob_magic, DEBUG: Пре-фит модели
24-Apr-20 14:30:44, sponge_bob_magic, DEBUG: -------------
24

In [11]:
scenario.experiment.pandas_df

,HitRate@10,NDCG@3,NDCG@5,NDCG@10,Surprisal@3,Surprisal@5,Surprisal@10
LightFMWrap(rank=53),0.786,0.298056,0.274744,0.267312,0.249019,0.255009,0.266485
LightFMWrap(rank=82),0.770,0.305654,0.282602,0.270065,0.248869,0.254147,0.264822


### Получение рекомендаций <a name="predict-scenario"></a>

In [12]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

24-Apr-20 14:31:34, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
24-Apr-20 14:31:34, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
24-Apr-20 14:31:34, sponge_bob_magic, DEBUG: Построение модели LightFM
/home/boris/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning:

pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead

24-Apr-20 14:31:36, sponge_bob_magic, DEBUG: Выделение дефолтных юзеров
24-Apr-20 14:31:36, sponge_bob_magic, DEBUG: Выделение дефолтных юзеров


In [13]:
recs.show()

+-------+-------+------------------+
|user_id|item_id|         relevance|
+-------+-------+------------------+
|    296|    129|               0.0|
|    296|    116|               0.0|
|    296|    283|               0.0|
|    296|    479|               0.0|
|    296|     25|               0.0|
|    296|    125|               0.0|
|    296|      7|               0.0|
|    296|    124|               0.0|
|    296|    475|               0.0|
|    296|    544|               0.0|
|    467|    250|1.5953795909881592|
|    467|    129|1.5653033256530762|
|    467|    324|1.3709657192230225|
|    467|    508| 1.199501872062683|
|    467|    325|0.9729878902435303|
|    467|   1067|0.9186442494392395|
|    467|    544|0.8913424015045166|
|    467|    346|0.8708140254020691|
|    467|    121|0.8460487723350525|
|    467|    460|0.8171349167823792|
+-------+-------+------------------+
only showing top 20 rows

